In [ ]:
import numpy as np
import pandas

from graphing.sampling_graphs import create_sample_distribution_graph_groupings, create_grouping_influence_graph, create_sample_distribution_graph
from learning.grouped_learning import Berta
from learning.grouped_linear_regression import Gustav
from model.datasets import Datasets, SynDataset
from model.synth_data_factory import SyntheticDataBuilder
from sampling.from_tests_sampler import FromTestsSamplingStrategy
from sampling.group_sampling.hamming_group_sampling_strategy import HammingGroupSamplingStrategy
from sampling.random_sampling.better_random_sampling_strategy import BetterRandomSamplingStrategy
from sampling.sampler import Sampler
from testing.grouped_tester import GroupedTester
from testing.grouped_tester_flipping import GroupedTesterSigns
from testing.tester import Tester
from util.util import get_samples_with_results, get_regression_metricts, make_df_representation
from sampling.group_sampling.mutex_aware_group_sampling_strategy import MutexAwareGroupSamplingStrategy
from util.util import get_samples_with_results_full

from sklearn import linear_model
pandas.set_option("display.max_columns", None)
%load_ext autoreload
%autoreload 2

# Data Generation

In [ ]:
data_builder = SyntheticDataBuilder(
    parameters=10,
    influential_parameters=1,
    mutex_groups=[],
    mutex_required=[],
    constrains=0,
    interactions=1
)
data_builder.print_overview()
influential_params = [i - 1 for i in data_builder.influential_features]
vm, test_strategy = data_builder.get_vm(), data_builder.get_testing_strategy()
vm, test_strategy = Datasets().get_dataset('javagc')


In [ ]:
tester = Tester(test_strategy)
sampler_test = Sampler(FromTestsSamplingStrategy(vm, test_strategy))
#sampler_test = Sampler(BetterRandomSamplingStrategy(vm))

x_test, y_test, samples_test = get_samples_with_results_full(sampler_test, tester, 100)
create_sample_distribution_graph(samples_test, vm)

# Sampling

In [ ]:
sampler = Sampler(BetterRandomSamplingStrategy(vm))
x, y, samples = get_samples_with_results_full(sampler_test, tester, 100)
#display((make_df_representation(x, y)))sampler.reset()
create_sample_distribution_graph(samples, vm)

In [ ]:
clf = linear_model.LinearRegression()
clf.fit(x, y)
y_predicted = clf.predict(x_test)
metrics = get_regression_metricts(y_test, y_predicted)

print(metrics)

In [ ]:
coef = pandas.DataFrame( clf.coef_)
ax = coef.plot.bar()
ax.set_xticklabels(vm.get_features())
print(clf.intercept_)

In [ ]:
df = pandas.read_csv('./resources/javagc_measurements.csv', delimiter=";")
display(df)
df['performance'] = df['performance'] / 30
df[['performance','NewRatio_32','SurvivorRatio_1']][5000:8000].plot()